# PyTorch Example

#### Using King County Housing Data

In [3]:
import pandas as pd
import numpy as np

In [135]:
import torch

In [5]:
df = pd.read_csv('../src/kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [179]:
cols = ['sqft_living', 'sqft_lot', 'sqft_living15', 'price']
data = df[cols][:1000]

In [180]:
x = [[data['sqft_living'][i], data['sqft_lot'][i], data['sqft_living15'][i]] for i in range(len(data))]

In [181]:
x[:10]

[[1180, 5650, 1340],
 [2570, 7242, 1690],
 [770, 10000, 2720],
 [1960, 5000, 1360],
 [1680, 8080, 1800],
 [5420, 101930, 4760],
 [1715, 6819, 2238],
 [1060, 9711, 1650],
 [1780, 7470, 1780],
 [1890, 6560, 2390]]

In [182]:
x = np.array(x)

In [183]:
x

array([[ 1180,  5650,  1340],
       [ 2570,  7242,  1690],
       [  770, 10000,  2720],
       ...,
       [ 1030,  4188,  1450],
       [ 1850,  9550,  2250],
       [ 1830,  4590,  1650]])

In [184]:
inputs = torch.from_numpy(x)

In [185]:
inputs = inputs.type(torch.float)

In [186]:
inputs.dtype

torch.float32

In [187]:
inputs.shape

torch.Size([1000, 3])

In [188]:
y = [[data['price'][i]] for i in range(len(data))]

In [189]:
y = np.array(y)

In [190]:
targets = torch.from_numpy(y)

In [191]:
targets.shape

torch.Size([1000, 1])

In [192]:
# req_Grad signals to autograd that every operation on them should 
# be tracked.

In [236]:
weights = torch.rand(1, 3, requires_grad=True)
biases = torch.rand(1, requires_grad=True)
print(weights, biases)

tensor([[0.6756, 0.4240, 0.6770]], requires_grad=True) tensor([0.6039], requires_grad=True)


In [237]:
weights.requires_grad

True

In [238]:
weights.retain_grad()
biases.retain_grad()

In [239]:
weights.shape

torch.Size([1, 3])

In [240]:
weights.dtype

torch.float32

In [241]:
def model(x):
    return x.long() @ weights.t().long() + biases

In [242]:
predictions = model(inputs)

In [243]:
predictions.shape

torch.Size([1000, 1])

In [244]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [245]:
loss = mse(predictions, targets)
print(loss)

tensor(3.8543e+11, dtype=torch.float64, grad_fn=<DivBackward0>)


In [246]:
torch.__version__

'1.7.1'

In [247]:
loss.backward()

In [248]:
print(biases)
print(biases.grad)

tensor([0.6039], requires_grad=True)
tensor([-1040164.5000])


In [249]:
print(weights)

tensor([[0.6756, 0.4240, 0.6770]], requires_grad=True)


In [233]:
weights.grad = torch.rand(1,3)

In [232]:
print(weights.grad)

None


In [250]:
for i in range(100):
    predictions = model(inputs)
    loss = mse(predictions, targets)
    loss.backward()
    with torch.no_grad():
        weights -= weights.grad * 1e-5
        biases -= biases.grad * 1e-5
        weights.grad.zero()
        biases.grad.zero()

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

In [ ]:
#remember each element in the prediction differs from actual target by the
# SQRT of the loss

# If the gradient element is POSITIVE:
# increasing the element's value slightly will increase the loss
# decreasing the element's value slightly will decrease the loss

# if the gradient element is NEGATIVE
# increasing the element's value slightly will decrease the loss
# decreasing the element's value slightly will increase the loss

In [ ]:
# Numeric operations example